In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.sparse.linalg import svds

In [262]:
df = pd.read_csv("data/real.csv",index_col=0)

In [263]:
df = df.iloc[0:,[0,1,2]]

In [264]:
df.head()

,Store,User,Rating
0,NAP's Coffee & Roasters,Nuunet D.,5
1,NAP's Coffee & Roasters,vanai sun,3
2,NAP's Coffee & Roasters,Phonanan Seeon,5
3,NAP's Coffee & Roasters,Chavakorn Laophetsakunchai,5
4,NAP's Coffee & Roasters,community man,5


In [265]:
df.shape

(3291, 3)

In [266]:
data = df.copy()
data = data.drop_duplicates(['Store','User'])
data = data.dropna()


# data = df.pivot(index='User',columns='Store',values='Rating')

In [267]:
data.shape

(3285, 3)

In [268]:
pivot_data = data.pivot(index='User',columns='Store',values='Rating')
pivot_data.head(5)

Store,11.11 Gallery and Coffee,Abe Specialty Coffee,Amarna,Anna Roasters,Attaroast,BaanHuakham Cafe & Farmstay,BalconyKiss Coffee,Blendstorm Coffee Roasters,Blues Coffee,Bossa cafe,...,ROSIEHOLM,Rogue Roasters,Saereesook,Snoopcat Cafe,SongSarn,Stufe coffee,Tree Cafe Rim Moon,Yuanjai Cafe,r o u n d,sangob
User,,,,,,,,,,,,,,,,,,,,,
01 มติมนต์ จํานงค์,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
113_kammika raksa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1904x,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984whatthehell,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212ปาณิตตา ศิริชัย,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Split data

In [269]:
from sklearn.model_selection import train_test_split

In [333]:
user2idx = {user: idx for idx, user in enumerate(data.User.unique())}
idx2user = {idx: user for user, idx in user2idx.items()}

item2idx = {item: idx for idx, item in enumerate(data.Store.unique())}
idx2item = {idx: item for item, idx in item2idx.items()}

In [334]:
encoded_df = data.copy()

encoded_df["User"] = encoded_df["User"].map(user2idx)
encoded_df["Store"] = encoded_df["Store"].map(item2idx)

gg = pd.pivot_table(encoded_df, values='Rating', index='User', columns='Store')
gg.head(5)

Store,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
User,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,NaN,5.0,NaN,5.0,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [272]:
train_df = encoded_df.copy().sample(frac=0.6, random_state=1)

test_df = encoded_df.copy().drop(train_df.index)

In [57]:
train = gg.sample(frac=0.8, random_state=1).fillna(0)
test = gg.drop(train.index).fillna(0)

In [282]:
train

Store,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
User,,,,,,,,,,,,,,,,,,,,,
1870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
1447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
535,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
1953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [280]:
train_df.pivot(index='User',columns='Store',values='Rating')

Store,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
User,,,,,,,,,,,,,,,,,,,,,
0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [283]:
U, sigma, Vt = svds(train.values, k = NUMBER_OF_FACTORS_MF)

In [284]:
U

In [83]:
# Compute the mean of each column in the training data
train_mean = train.mean()

# Perform mean normalization on training and testing data
train_norm = train - train_mean
test_norm = test - train_mean

# Apply mean normalization to train_mean
train_mean_norm = train_mean - train_mean

# Apply SVD on the normalized training data
U, sigma, Vt = svds(train_norm.values, k=15)

# Reconstruct the matrix using the SVD components and the normalized mean
matrix_reconstructed = U.dot(np.diag(sigma)).dot(Vt) + train_mean_norm.values.reshape(1,-1)

# Convert the reconstructed matrix back to its original scale
matrix_reconstructed_original_scale = matrix_reconstructed + train_mean.values.reshape(1,-1)


In [90]:
pd.DataFrame(matrix_reconstructed_original_scale,index=train.index,columns=train.columns)


Store,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
User,,,,,,,,,,,,,,,,,,,,,
1870,-0.003622,0.204977,0.070842,-0.003192,-0.321138,-0.032927,0.003331,-0.007809,0.069446,0.046401,...,0.024369,0.119780,-0.005776,0.138817,0.021627,-0.016281,3.938044,0.265043,0.015017,0.007726
1447,-0.000372,0.007087,0.013300,-0.001415,-0.022982,-0.011079,-0.005063,0.000805,-0.029609,-0.010526,...,-0.000225,0.005603,-0.006640,0.110594,0.006727,0.037573,-0.006737,0.027110,0.001644,-0.000136
1138,0.012999,0.013444,-0.051417,0.011215,-0.004404,0.018451,0.021001,0.021787,-0.001796,0.001153,...,0.025398,-0.003856,0.025916,-0.013376,0.012764,-0.037634,0.024114,0.088081,0.014666,0.078327
535,0.045643,0.490584,0.650879,-0.007446,2.990069,-0.143804,0.052833,-0.065951,0.223315,0.060927,...,0.169378,0.142051,0.103908,0.637080,0.046530,-0.117059,-0.259969,1.912992,0.108437,0.558940
1577,-0.001770,-0.083674,-0.014371,-0.008049,0.028539,-0.002187,-0.011113,-0.007682,0.023880,-0.007922,...,4.985795,-0.029406,-0.006721,0.033310,0.001474,0.023228,-0.004134,-0.052237,-0.006339,-0.042486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722,-0.003881,-0.067749,-0.092985,-0.008081,0.060759,-0.008342,-0.016774,-0.016336,0.021843,-0.038460,...,2.976672,-0.016766,-0.026167,-0.006336,-0.003796,0.037804,-0.008130,-0.068985,4.983784,-0.109999
1953,0.007677,0.057711,-0.041835,0.009489,-0.032240,-0.000269,0.013008,0.008368,-0.008348,-0.001175,...,0.016778,-0.023141,-0.003865,-0.031085,0.025894,0.014600,0.007733,0.117374,4.007277,0.024172
836,0.010087,-0.056651,0.047205,-0.002385,-0.190800,-0.018562,0.000242,4.977860,0.001913,-0.003982,...,0.008308,-0.061205,-0.006453,-0.029249,0.012528,0.021832,-0.024369,0.218211,0.003136,0.006231


In [67]:
matrix_reconstructed.shape

(1790, 33)

## Teset min max norm

In [189]:
NUMBER_OF_FACTORS_MF = 5
#Performs matrix factorization of the original user item matrix
#U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)
U, sigma, Vt = svds(train.values, k = NUMBER_OF_FACTORS_MF)

In [190]:
sigma = np.diag(sigma)
sigma

array([[59.59075397,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , 62.48973936,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , 63.75725447,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , 71.70312512,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , 85.13802374]])

In [191]:
U

array([[-0.0039267 , -0.0023296 , -0.00063913,  0.00173602,  0.00342841],
       [-0.01077257,  0.00978494,  0.07091025,  0.01526732,  0.01457634],
       [ 0.00308665,  0.08634596, -0.01696363,  0.00589812,  0.01127513],
       ...,
       [-0.00276135, -0.00063716,  0.00143804,  0.00281169,  0.00664023],
       [-0.0044406 , -0.00247208, -0.0039532 ,  0.00881048,  0.00791743],
       [-0.05296583, -0.01043139, -0.02481789,  0.02644603,  0.01684675]])

In [192]:
Vt

array([[-1.69416178e-01, -3.37127507e-02, -2.29805958e-02,
         7.25220777e-01, -5.72150878e-02,  2.13254963e-02,
        -6.31254689e-01, -3.29102167e-02,  5.05841241e-03,
        -1.32746623e-03, -3.47769175e-02, -2.60970593e-02,
        -5.15775455e-03,  2.71209138e-03, -1.57992469e-02,
         1.08527273e-03,  2.29919986e-02,  2.29919986e-02,
        -6.35527512e-02, -8.96992699e-03, -1.81468041e-02,
        -1.28389080e-01, -4.23001202e-03,  5.77155598e-03,
        -1.31008939e-02,  1.22304382e-03, -3.17186535e-02,
         4.27841998e-03, -9.28695187e-02, -5.84988056e-02,
        -8.30139974e-03,  5.32579812e-02,  1.09235605e-03],
       [-1.36505456e-01, -6.64805172e-03, -8.07691941e-03,
        -1.63525068e-01,  1.21107584e-02,  1.69172410e-02,
        -1.30370999e-01, -7.96324263e-03, -5.79458016e-03,
        -3.19741484e-03, -3.07393712e-03, -1.00390124e-03,
         3.21223682e-02,  3.69379165e-03,  5.68311003e-03,
        -1.90342821e-03,  6.74467037e-01,  6.74467037e-

In [181]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[-4.03822957e-03,  2.22295173e-01,  8.50449394e-02, ...,
         2.23055323e-01, -4.64004621e-03, -4.16384737e-02],
       [ 2.67315561e-04, -3.35507664e-03,  7.00341263e-03, ...,
         3.65316135e-02, -1.25092768e-03, -2.81977838e-02],
       [ 5.90181974e-03, -1.84867923e-02, -6.18806854e-02, ...,
         1.04064450e-04, -3.45117362e-04,  3.15387018e-02],
       ...,
       [ 7.34573217e-03, -7.38862738e-02,  5.08947278e-02, ...,
         1.06490584e-01, -3.22282810e-03, -3.62082840e-02],
       [ 4.02825300e-02,  4.98658198e-01,  6.91139061e-01, ...,
        -1.19039656e-02, -4.09447674e-02,  2.79743110e-01],
       [-3.10921185e-03, -5.18922594e-02, -8.71410316e-02, ...,
        -2.71423006e-02,  3.98625204e-03, -1.43512833e-02]])

In [182]:
all_user_predicted_ratings_norm = (all_user_predicted_ratings - all_user_predicted_ratings.min())/ (all_user_predicted_ratings.max() - all_user_predicted_ratings.min())

In [183]:
pd.DataFrame(all_user_predicted_ratings_norm,index=train.index,columns=train.columns)

Store,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
User,,,,,,,,,,,,,,,,,,,,,
1870,0.273740,0.301231,0.284560,0.272374,0.221803,0.269911,0.274759,0.270697,0.283143,0.279253,...,0.274674,0.292573,0.271996,0.289934,0.274373,0.267423,0.746454,0.301323,0.273667,0.269174
1447,0.274263,0.273823,0.275082,0.274002,0.270192,0.272859,0.273634,0.273869,0.270839,0.272215,...,0.273922,0.276257,0.272728,0.285790,0.273730,0.279394,0.272926,0.278668,0.274079,0.270806
1138,0.274948,0.271986,0.266715,0.274302,0.275774,0.275723,0.275147,0.275030,0.274796,0.272766,...,0.274559,0.275600,0.276404,0.273381,0.273659,0.268993,0.274924,0.274244,0.274189,0.278062
535,0.276493,0.346169,0.369691,0.262138,0.581262,0.254373,0.277374,0.251837,0.304960,0.282435,...,0.276914,0.300812,0.283316,0.365252,0.277174,0.222897,0.208696,0.428346,0.272453,0.324352
1577,0.274463,0.264895,0.272245,0.273932,0.276914,0.274369,0.273594,0.274038,0.276866,0.273100,...,0.881038,0.271561,0.273202,0.277227,0.273568,0.277915,0.274785,0.273749,0.274300,0.269559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1722,0.274794,0.272236,0.265485,0.274326,0.274062,0.274090,0.274333,0.273724,0.275902,0.271315,...,0.638384,0.271599,0.271717,0.272496,0.275179,0.277364,0.273859,0.277203,0.881026,0.266764
1953,0.274570,0.277117,0.268187,0.274451,0.272808,0.274052,0.274618,0.273918,0.274303,0.272441,...,0.274286,0.273407,0.272714,0.271405,0.275308,0.274969,0.273667,0.276840,0.759634,0.270500
836,0.275123,0.265257,0.280413,0.273240,0.251837,0.272523,0.273675,0.878120,0.275579,0.272339,...,0.274038,0.269471,0.272447,0.273520,0.273478,0.273690,0.269814,0.287165,0.273840,0.269833


In [184]:
mse = ((all_user_predicted_ratings_norm - train.values)**2).mean()
mse

0.7318633303505417

In [285]:
train = gg.sample(frac=0.8, random_state=1).fillna(0)
test = gg.drop(train.index).fillna(0)

U, sigma, Vt = svds(train.values,k=15)
matrix_reconstructed = U.dot(np.diag(sigma)).dot(Vt)

In [259]:
test1 = all_user_predicted_ratings_norm[5]
test2 = all_user_predicted_ratings_norm[1]

In [260]:
cosine_similarity(test1,test2)

0.8839525481361726

In [261]:
test1

array([0.27728594, 0.29723781, 0.31349659, 0.89121846, 0.32522544,
       0.28120016, 0.27930211, 0.29132715, 0.30673015, 0.28000169,
       0.33019298, 0.30594097, 0.31046031, 0.27855525, 0.27139399,
       0.27793946, 0.27551941, 0.27551941, 0.27728218, 0.33975765,
       0.31012299, 0.26445224, 0.27639863, 0.26984655, 0.28864466,
       0.28191411, 0.33592822, 0.27693512, 0.34564428, 0.28381239,
       0.2385466 , 0.27773904, 0.30837513])

In [220]:

def cosine_similarity(a, b):
    '''
    This function will calculate the cosine similarity between two vectors
    '''
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

def get_similarities(mat, id_):
    '''
    This function will use the cosine similarity function to generate a similarity 
    dictionary assocaited to an id the user passes in. The similarity dictionary will
    have the ids as the keys and the similarity in comparison to the user input id as
    the values.
    
    params:
        mat (List -> List) : A 2-D array assocaited to either the user / item matrix
                             after SVD
        id_ (Integer) : The id of the user / item you want to find similarities for.
                        The id must be in the range of the input matrix shape.
    
    returns:
        This function will return the similarity dictionary ordered by the values in 
        descending order.
        
    example:
        mat = np.asarray([
            [2,3,4],
            [6,5,3],
            [5,3,2]
        ])
        id_ = 2
        get_similarities(mat, id_)
    '''
    # create similarity hashmap, keys are ids and values are similarities
    sim_dct = {} 
    for col in range(0, mat.shape[1]):
        sim = cosine_similarity(mat[:,id_], mat[:,col])
        sim_dct[col] = sim
    
    # sort dictionary based on similarities 
    sim_dct = {k: v for k, v in sorted(sim_dct.items(), key=lambda item: item[1], reverse = True)}
    return sim_dct

In [196]:
def recommend(mat, id_, n_recs):
    '''
    This function will get the top n recommendations assocaited to an id.
    
    params:
        mat (List -> List) : A 2-D array assocaited to either the user / item matrix
                             after SVD
        id_ (Integer) : The id of the user / item you want to find similarities for.
                        The id must be in the range of the input matrix shape.
        n_recs (Integer) : The number of recommendations you want.
        
    returns:
        This function will return a list of ids most similar to the input id you passed.
    '''
    sim_dct = get_similarities(mat, id_)
    similar_ids = list(sim_dct.keys())[1:n_recs+1]
    return similar_ids

In [200]:
id_ = 8
n_recs = 10
print(get_similarities(U, id_))

IndexError: index 8 is out of bounds for axis 1 with size 5

In [ ]:
np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [289]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# # Load the dataset
# data = np.loadtxt('data.csv', delimiter=',')

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(data[:, :-1], data[:, -1], test_size=0.3, random_state=42)

# Perform SVD
svd = TruncatedSVD(n_components=10)
X_train_svd = svd.fit_transform(train)
X_test_svd = svd.transform(test)

# Fit a model
model = LinearRegression()
model.fit(X_train_svd, y_train)

# Evaluate the model
y_pred = model.predict(X_test_svd)
mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)

NameError: name 'y_train' is not defined

In [303]:
train_df.iloc[:,1]

256      236
582      367
3243    2165
935      774
1164     932
        ... 
1381    1053
1658    1169
1829     592
3039    2035
34        34
Name: User, Length: 1971, dtype: int64

In [305]:
n_users = np.max(train_df.iloc[:, 1])
n_items = np.max(train_df.iloc[:, 0])

In [314]:
train_matrix = np.zeros((n_users, n_items))
for line in train_df:
    train_matrix[int(line[0])-1, int(line[1])-1] = line[2]

Store


ValueError: invalid literal for int() with base 10: 'S'

In [320]:
train_data

,Store,User,Rating
2537,ROSIEHOLM,Suchada Rungkijvarasathien,4
1112,ROOF COFFEE,iDEA,5
1613,Commune Drink/Talk/Share,Parichat Kamutchat,5
658,BalconyKiss Coffee,LoBlack vlog,4
1429,Blues Coffee,KKJ BABY TV,3
...,...,...,...
1097,ROOF COFFEE,บักหัวล้าน อีหน้าโง่,1
1132,11.11 Gallery and Coffee,Mewwy Naruedee,5
1296,BaanHuakham Cafe & Farmstay,เปาเปา Racing Shop อุบล,5
861,Anna Roasters,Tanathon Boonla,4


In [360]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error

# Load the dataset


# Split the dataset into training and testing sets
train_data, test_data = train_test_split(encoded_df.copy(), test_size=0.3, random_state=42)

# Create a user-item matrix
train_matrix = train_data.pivot(index='User', columns='Store', values='Rating').fillna(0)
n_users, n_items = train_matrix.shape

# Perform SVD
u, s, vt = svds(train_matrix.values, k=15)
s_diag_matrix = np.diag(s)

# Predict ratings
user_ratings = np.dot(np.dot(u, s_diag_matrix), vt)
test_predictions = []
for index, row in test_data.iterrows():
    user_id = row['User']
    item_id = row['Store']
    if user_id < n_users and item_id < n_items:
        rating = user_ratings[user_id, item_id]
    else:
        rating = 0
    test_predictions.append(rating)

# Evaluate the model
test_ratings = test_data['Rating']
mse = mean_squared_error(test_ratings, test_predictions)
print("MSE: ", mse)


MSE:  18.69117807025591


In [367]:
test_ratings

2214    3
2837    5
1659    5
1655    5
1453    5
       ..
3041    4
1227    5
3351    5
2471    4
2614    5
Name: Rating, Length: 986, dtype: int64

In [338]:
train_matrix

Store,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
User,,,,,,,,,,,,,,,,,,,,,
1,3.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,5.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
